<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/Tripadvisor_July_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
!pip install "httpx[http2,brotli]" parsel
!pip install loguru

# import modules
import re
import httpx
import json
import random
import string
import difflib
import math
import numpy as np
import requests
import pandas as pd
from google.colab import files
from loguru import logger as log

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.3 MB/s eta 0:00:00


In [2]:
# To avoid being instantly blocked we'll be using request headers that
# mimic Chrome browser on Windows
BASE_HEADERS = {
    "authority": "www.tripadvisor.com",
    "accept-language": "en-US,en;q=0.9",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-language": "en-US;en;q=0.9",
    "accept-encoding": "gzip, deflate, br",
}
# start HTTP session client with our headers and HTTP2
client = httpx.AsyncClient(
    http2=True,  # http2 connections are significantly less likely to get blocked
    headers=BASE_HEADERS,
    timeout=httpx.Timeout(15.0),
    limits=httpx.Limits(max_connections=5),
)

In [8]:
payload = [
    {
        "variables":{
            "request":{
                "query":"banjarmasin",
                "limit":10,
                "scope":"WORLDWIDE",
                "locale":"en-US",
                "scopeGeoId":1,
                "searchCenter":None,
                "types":["LOCATION","RESCUE_RESULT"],
                "locationTypes":["GEO","NEIGHBORHOOD","ACCOMMODATION"],
                "userId":None,
                "context":{
                    "searchSessionId":"0002d1b71c31f8cc.ssid",
                    "typeaheadId":"1752215195697",
                    "uiOrigin":"homepage_faceted_search_HOTELS",
                    "routeUid":"c50707db-8d80-4795-b53f-b37a5b036f26"
                    },
                "enabledFeatures":[],
                "includeRecent":True,
                "includeNestedResults":True
                }
            },
        "extensions":{
            "preRegisteredQueryId":"c2e5695e939386e4"
            }
        }
    ]

In [ ]:
# we need to generate a random request ID for this request to succeed
random_request_id = "".join(
    random.choice(string.ascii_lowercase + string.digits) for i in range(180)
)

# set the headers
headers = {
    "X-Requested-By": random_request_id,
    "Referer": "https://www.tripadvisor.com/",
    "Origin": "https://www.tripadvisor.com",
}

# get the page with method POST and pass the payload and headers as the parameters
result = await client.post(
    url="https://www.tripadvisor.com/data/graphql/ids",
    json=payload,
    headers=headers,
)
data = json.loads(result.content)
data

In [5]:
data[0]["data"]["Typeahead_autocomplete"]["results"][0]

{'__typename': 'Typeahead_LocationItem',
 'documentId': 'loc;303950;g303950',
 'sponsored': False,
 'locationId': 303950,
 'details': {'localizedName': 'Banjarmasin',
  'localizedAdditionalNames': {'longOnlyHierarchy': 'South Kalimantan, Kalimantan, Indonesia, Asia'},
  'streetAddress': {'street1': ''},
  'locationV2': {'placeType': 'CITY',
   'hierarchy': {'parentId': 2301801,
    'parent': {'names': {'longOnlyHierarchyTypeaheadV2': 'Kalimantan, Indonesia'}},
    'naturalParentId': 294225,
    'naturalParent': {'names': {'name': 'Indonesia'}, 'placeType': 'COUNTRY'}},
   'names': {'longOnlyHierarchyTypeaheadV2': 'Kalimantan, Indonesia'},
   'vacationRentalsRoute': {'url': '/VacationRentals-g303950-Reviews-Banjarmasin_South_Kalimantan_Kalimantan-Vacation_Rentals.html'}},
  'url': '/Tourism-g303950-Banjarmasin_South_Kalimantan_Kalimantan-Vacations.html',
  'HOTELS_URL': '/Hotels-g303950-Banjarmasin_South_Kalimantan_Kalimantan-Hotels.html',
  'ATTRACTIONS_URL': '/Attractions-g303950-Acti

In [7]:
payload2 = [
    {
        "variables": {
            "geoId": 303950,
            "blenderId": None,
            "boundingBox": None,
            "centerAndRadius": None,
            "travelInfo": None,
            "currency": "USD",
            "pricingMode": None,
            "filters": {
                "selectTravelersChoiceWinner": False,
                "selectTravelersChoiceBOTBWinner": False,
                "minRating": None,
                "neighborhoodsOrNear": None,
                "priceRange": None,
                "amenities": None,
                "brands": None,
                "classes": None,
                "styles": None,
                "hoteltypes": None,
                "categories": None,
                "anyTags": None,
                "hotelowners": None,
            },
            "offset": 0,
            "limit": 30,
            "sort": "BEST_VALUE",
            "clientType": "DESKTOP",
            "isMapView": False,
            "viewType": "LIST",
            "productId": "Hotels",
            "pageviewId": "33333368-36d5-4c13-8cda-64203e354821",
            "sessionId": "8BE87E0515EE5227AC3E37868B6E4B0F",
            "route": {
                "page": "HotelsFusion",
                "params": {
                    "geoId": 303950,
                    "contentType": "hotel",
                    "webVariant": "HotelsFusion",
                },
            },
            "userEngagedFilters": False,
            "loadPoiThumbnail": False,
            "loadLocationSEOData": True,
            "loadLocationInfoData": False,
            "loadNearbyPointOfInterestPlaceType": False,
            "metaMarketingQueryString": "",
            "loadReviewSubratingAvgs": False,
            "isDiscoEligible": True,
            "resolveGeoIdFromBoundingBox": False,
            "polling": False,
            "tertiaryOffers": False,
            "includePhotoSizes": False,
            "requestNumber": 0,
        },
        "extensions": {"preRegisteredQueryId": "3166880147a6527c"},
    }
]

In [ ]:
# we need to generate a random request ID for this request to succeed
random_request_id = "".join(
    random.choice(string.ascii_lowercase + string.digits) for i in range(180)
)

# set the headers
headers = {
    "X-Requested-By": random_request_id,
    "Referer": "https://www.tripadvisor.com/",
    "Origin": "https://www.tripadvisor.com",
}

# get the page with method POST and pass the payload and headers as the parameters
result = await client.post(
    url="https://www.tripadvisor.com/data/graphql/ids",
    json=payload2,
    headers=headers,
)
data = json.loads(result.content)
data

In [ ]:
data[0]['data']['list']['results']

In [11]:
payload3 = [
    {
        "variables": {
            "locationId": 1650526,
            "filters": [{"axis": "LANGUAGE", "selections": ["en"]}],
            "limit": 10,
            "offset": 10,
            "sortType": None,
            "sortBy": "SERVER_DETERMINED",
            "language": "en",
            "useAwsTips": True,
        },
        "extensions": {"preRegisteredQueryId": "51c593cb61092fe5"},
    }
]

In [13]:
# we need to generate a random request ID for this request to succeed
random_request_id = "".join(
    random.choice(string.ascii_lowercase + string.digits) for i in range(180)
)

# set the headers
headers = {
    "X-Requested-By": random_request_id,
    "Referer": "https://www.tripadvisor.com/",
    "Origin": "https://www.tripadvisor.com",
}

# get the page with method POST and pass the payload and headers as the parameters
result = await client.post(
    url="https://www.tripadvisor.com/data/graphql/ids",
    json=payload3,
    headers=headers,
)
data = json.loads(result.content)
data[0]['data']['ReviewsProxy_getReviewListPageForLocation'][0]['reviews']

[{'id': 1016426236,
  'status': 'PUBLISHED',
  'reviewDetailPageWrapper': {'reviewDetailPageRoute': {'url': '/ShowUserReviews-g303950-d1650526-r1016426236-Rattan_Inn-Banjarmasin_South_Kalimantan_Kalimantan.html'}},
  'location': {'locationId': 1650526,
   'name': 'Rattan Inn',
   'placeType': 'ACCOMMODATION',
   'businessAdvantageData': None},
  'createdDate': '2025-07-03',
  'publishedDate': '2025-07-03',
  'userProfile': {'id': '494C77B762F7239E0C0C1200D0C3A4A8',
   'isVerified': False,
   'displayName': 'Paradise11683127058',
   'username': 'Paradise11683127058',
   'avatar': {'__typename': 'MemberProfile_MediaId',
    'data': {'__typename': 'Media_PhotoResult',
     'id': 452389009,
     'photoSizeDynamic': {'urlTemplate': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/1a/f6/e8/91/default-avatar-2020-62.jpg?w={width}&h={height}&s=1',
      'maxHeight': 1200,
      'maxWidth': 1200}}},
   'hometown': {'locationId': None, 'fallbackString': None, 'location': None},
   'route